In [67]:
import ipywidgets as widgets
from IPython import display
import pandas as pd
import numpy as np
from random import shuffle
from copy import deepcopy

In [68]:
players = [
    #'Austin',
    #'Ben',
    'Brady',
    #'Bradley',
    'Bryce',
    'Crosley',
    'Ethan',
    'Jake V.',
    'Jacob M.',
    'Noah',
]

positions = [
    'P',
    '1B',
    'SS',
    '2B',
    '3B',
    'CF',
    'LF',
    'RF',
    'LC',
    'RC',
]

if len(players) < 8:
    positions[5] = 'LC'
    positions[6] = 'RC'

if len(players) < 7:
    positions[5] = 'CF'

positions = positions[:len(players)]

In [69]:
shuffle(players)
print(players)
print(positions)

['Brady', 'Crosley', 'Noah', 'Jacob M.', 'Bryce', 'Jake V.', 'Ethan']
['P', '1B', 'SS', '2B', '3B', 'LC', 'RC']


In [86]:
innings = 3
def random_lineup():
    field, hitting = [], []
    for inning in range(innings):
        index = positions
        shuffle(players)
        field.append(deepcopy(players))
        shuffle(players)
        hitting.append(deepcopy(players))
    return list(zip(*field)), list(zip(*hitting))
bad = True
from collections import Counter
while bad:
    field, hitting = random_lineup()
    bad = False
    # same person doesn't bat in the same spot or play the same position twice
    if any(filter(lambda x: len(set(x))<3, field+hitting)):
        bad = True
        continue
    # at least 6 different people hit 1st or last
    leadoff = set(hitting[0])
    hr_hitter = set(hitting[-1])
    good_hitting = leadoff | hr_hitter
    if len(good_hitting)<min(6, len(players)):
        bad = True
        continue
    # at least 6 different people are pitcher or 1st
    pitcher = set(field[0])
    first_base = set(field[1])
    if len(pitcher | first_base)<min(6, len(players)):
        bad = True
        continue
    # at least 6 different people are pitcher or leadoff
    if len(pitcher | leadoff)<min(6, len(players)):
        bad = True
        continue
    # At least 9 unique people are lead-off and pitcher/1st
    good_fielding = pitcher | first_base
    if len(good_fielding | leadoff)<min(9, len(players)):
        bad = True
        continue
    # Everyone gets at least 1 of the good spots:
    if len(good_fielding | good_hitting)<min(10, len(players)):
        bad = True
    outfield = [p for f in field[5:] for p in f]
    c = Counter(outfield)
    if max(c.values())==3:
        bad = True
        continue
    # no one should get 3 infields if we have enough players
    if len(players) > 7:
        infield = [p for f in field[:5] for p in f]
        c = Counter(infield)
        if max(c.values())==3:
            bad = True
            continue
    else:
        # if only 7 players make sure no one gets outfield twice
        c = Counter(outfield)
        if max(c.values())>1:
            bad = True
            continue

df_f = pd.DataFrame(index=positions, columns=range(1,innings+1), data=field)
df_f.style.set_caption('Fielding')
df_h = pd.DataFrame(index=range(1, len(players)+1), columns=range(1,innings+1), data=hitting)
df_h.style.set_caption('Batting')

# create output widgets
widget1 = widgets.Output()
widget2 = widgets.Output()

# render in output widgets
with widget1:
    display.display(df_f)
with widget2:
    display.display(df_h)

# create HBox
hbox = widgets.HBox([widget1, widget2])

h = widgets.HTML(value='<b><h3>05/20/22 - Liberty Flames</h3></b>')


vbox = widgets.VBox()

vbox.children = [h, hbox]
vbox.layout.align_items = 'center'
vbox

In [72]:
df_f, df_h

(           1         2         3
 P      Bryce      Noah     Ethan
 1B   Crosley  Jacob M.     Brady
 SS     Brady   Jake V.     Bryce
 2B      Noah     Brady  Jacob M.
 3B     Ethan   Crosley   Jake V.
 LC  Jacob M.     Bryce      Noah
 RC   Jake V.     Ethan   Crosley,
           1         2         3
 1     Brady  Jacob M.   Jake V.
 2      Noah   Crosley     Brady
 3     Bryce   Jake V.   Crosley
 4  Jacob M.     Ethan      Noah
 5     Ethan     Brady  Jacob M.
 6   Jake V.      Noah     Bryce
 7   Crosley     Bryce     Ethan)